Mount Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Install

In [ ]:
!pip install -q torchaudio transformers datasets g2p_en

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 810.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 28.0 MB/s eta 0:00:00

In [ ]:
import torchaudio
from transformers import pipeline
import torch

## Input User Data

In [ ]:
audio_file = "/content/Voice.mp3"
waveform, sample_rate = torchaudio.load(audio_file)

In [ ]:
# from google.colab import files
# import torchaudio
# import ipywidgets as widgets
# from IPython.display import display

# # Upload audio file
# uploaded = files.upload()

# # Load the uploaded audio file
# audio_file = next(iter(uploaded.keys()))
# waveform, sample_rate = torchaudio.load(audio_file)

# # Button to confirm selection
# confirm_button = widgets.Button(description="Confirm Selection")

# # Display the button
# display(confirm_button)

## Load the Whisper Model

In [ ]:
# Set device
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load Whisper model for automatic speech recognition
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-base", device=device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Device set to use cpu


## Transcription Function

In [ ]:
def transcribe_audio(waveform, sample_rate):
    # Ensure the waveform is mono (1D)
    if waveform.size(0) > 1:  # If stereo, average the channels
        audio_input = waveform.mean(dim=0).numpy()
    else:
        audio_input = waveform.numpy()

    # Resample if needed (e.g., to 16000 Hz)
    desired_sample_rate = 16000
    if sample_rate != desired_sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=desired_sample_rate)
        audio_input = resampler(waveform).mean(dim=0).numpy()  # Resample and convert to 1D

    # Suppress deprecation warnings
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", FutureWarning)
        # Process the audio directly without `return_tensors`
        result = pipe(audio_input, return_timestamps=True)

    return result["text"]

# Example usage (assuming `waveform` and `sample_rate` are defined)
transcribed_text = transcribe_audio(waveform, sample_rate)
print("Transcribed Text:", transcribed_text)

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Transcribed Text:  Poor service, failed delivery, totally unprofessional, please avoid using this career service.
